In [72]:
#import data science libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#data types
import collections

#import general libraries
import gc
import time
import random
import datetime

#deep learning 
import tensorflow as tf

In [5]:
products = pd.read_csv('../data/products.csv')
print(products.shape)
products.head()

(49688, 4)


,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [6]:
aisles = pd.read_csv('../data/aisles.csv')
departments = pd.read_csv('../data/departments.csv')

In [7]:
orders = pd.read_csv('../data/orders.csv')
print(orders.shape)

(3421083, 7)


In [8]:
orders.head(12)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2539329,1,prior,1,2,8,NaN
1,2398795,1,prior,2,3,7,15.0
2,473747,1,prior,3,3,12,21.0
3,2254736,1,prior,4,4,7,29.0
4,431534,1,prior,5,4,15,28.0
5,3367565,1,prior,6,2,7,19.0
6,550135,1,prior,7,1,9,20.0
7,3108588,1,prior,8,1,14,14.0
8,2295261,1,prior,9,1,16,0.0
9,2550362,1,prior,10,4,8,30.0


In [9]:
orders_products_prior = pd.read_csv('../data/order_products__prior.csv')
print(orders_products_prior.shape)
orders_products_prior.head(5)

(32434489, 4)


,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [10]:
orders_products_prior_sample = orders_products_prior[orders_products_prior.order_id <= 1000000]
orders_products_prior_sample.shape

(9473192, 4)

### Merge order_products and orders dataset 

In [11]:
# MEGA DATASET
sample_join_orders = orders_products_prior_sample.set_index('order_id').join(orders.set_index('order_id'), how='left').reset_index()

In [12]:
print(sample_join_orders.shape)
sample_join_orders.head()

(9473192, 10)


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,2,33120,1,1,202279,prior,3,5,9,8.0
1,2,28985,2,1,202279,prior,3,5,9,8.0
2,2,9327,3,0,202279,prior,3,5,9,8.0
3,2,45918,4,1,202279,prior,3,5,9,8.0
4,2,30035,5,0,202279,prior,3,5,9,8.0


In [13]:
last_order_dict = (sample_join_orders
                   .groupby('order_id')
                   .agg({'add_to_cart_order':'max'})
                   .to_dict()['add_to_cart_order'])

In [14]:
sample_join_orders['last_order'] = sample_join_orders['order_id'].map(last_order_dict)
sample_join_orders['last_order_flag'] = (np.where(sample_join_orders['last_order'] == sample_join_orders['add_to_cart_order'],
                                         1,0))

In [15]:
print(sample_join_orders.shape)
sample_join_orders.head(10)

(9473192, 12)


,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order,last_order_flag
0,2,33120,1,1,202279,prior,3,5,9,8.0,9,0
1,2,28985,2,1,202279,prior,3,5,9,8.0,9,0
2,2,9327,3,0,202279,prior,3,5,9,8.0,9,0
3,2,45918,4,1,202279,prior,3,5,9,8.0,9,0
4,2,30035,5,0,202279,prior,3,5,9,8.0,9,0
5,2,17794,6,1,202279,prior,3,5,9,8.0,9,0
6,2,40141,7,1,202279,prior,3,5,9,8.0,9,0
7,2,1819,8,1,202279,prior,3,5,9,8.0,9,0
8,2,43668,9,0,202279,prior,3,5,9,8.0,9,1
9,3,33754,1,1,205970,prior,16,5,17,12.0,8,0


In [16]:
orders_products = sample_join_orders.merge(products, on='product_id', how='outer')
orders_products_aisles = orders_products.merge(aisles, on='aisle_id', how='outer')
orders_products_departments = orders_products_aisles.merge(departments, on='department_id', how='outer')

In [17]:
columns_df_product_embeddings = ['user_id','order_id','order_number','add_to_cart_order','product_id','product_name']

In [18]:
sorted_orders_products_departments = orders_products_departments.sort_values(['user_id',
                                                                              'order_number',
                                                                              'order_id',
                                                                              'add_to_cart_order'])

In [20]:
sorted_orders_products_departments.head()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,last_order,last_order_flag,product_name,aisle_id,department_id,aisle,department
7178054,473747.0,196,1.0,1.0,1.0,prior,3.0,3.0,12.0,21.0,5.0,0.0,Soda,77,7,soft drinks,beverages
6250952,473747.0,12427,2.0,1.0,1.0,prior,3.0,3.0,12.0,21.0,5.0,0.0,Original Beef Jerky,23,19,popcorn jerky,snacks
6302341,473747.0,10258,3.0,1.0,1.0,prior,3.0,3.0,12.0,21.0,5.0,0.0,Pistachios,117,19,nuts seeds dried fruit,snacks
1177272,473747.0,25133,4.0,0.0,1.0,prior,3.0,3.0,12.0,21.0,5.0,0.0,Organic String Cheese,21,16,packaged cheese,dairy eggs
4637501,473747.0,30450,5.0,0.0,1.0,prior,3.0,3.0,12.0,21.0,5.0,1.0,Creamy Almond Butter,88,13,spreads,pantry


In [21]:
df_product_embeddings = sorted_orders_products_departments[columns_df_product_embeddings]

In [22]:
df_product_embeddings.head(20)

,user_id,order_id,order_number,add_to_cart_order,product_id,product_name
7178054,1.0,473747.0,3.0,1.0,196,Soda
6250952,1.0,473747.0,3.0,2.0,12427,Original Beef Jerky
6302341,1.0,473747.0,3.0,3.0,10258,Pistachios
1177272,1.0,473747.0,3.0,4.0,25133,Organic String Cheese
4637501,1.0,473747.0,3.0,5.0,30450,Creamy Almond Butter
7177558,1.0,431534.0,5.0,1.0,196,Soda
6250870,1.0,431534.0,5.0,2.0,12427,Original Beef Jerky
6302319,1.0,431534.0,5.0,3.0,10258,Pistachios
1177189,1.0,431534.0,5.0,4.0,25133,Organic String Cheese
4060922,1.0,431534.0,5.0,5.0,10326,Organic Fuji Apples


In [97]:
pd.DataFrame(list_of_products).to_csv('subset_products_list_july30.csv')

In [23]:
#list of products in order
list_of_products = df_product_embeddings.product_name.tolist()

In [96]:
df_product_embeddings.to_csv('../data/subset_data_for_embeds_july30.csv', index=False)

In [24]:
#number of unique products in dataset
print(len(set(list_of_products)))
n_products = len(set(list_of_products))

49688


In [ ]:
# get sum and average for order_ids
# get sum and average for user_ids

In [ ]:
#build vanilla classifier

In [ ]:
#similarity between user and item (both being vectors)